In [1]:
# Import the library
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Import the MetaData file
# load file
MetaData = pd.read_csv("GSE103147_full_metadata.csv")
MetaData = MetaData.rename(columns={'Run': 'Sample_ID'})
MetaData.head(1)
MetaData.shape

,Unnamed: 0,geo_accession,title,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,...,BioSample,SampleType,TaxID,ScientificName,Tumor,CenterName,Submission,Consent,RunHash,ReadHash
0,GSM2754496,GSM2754496,100_04_0779_DAY_0_T_Ag85_100_L8.LB23,Public on Oct 17 2017,Aug 27 2017,Oct 17 2017,SRA,1,T cells,Homo sapiens,...,SAMN07564511,simple,9606,Homo sapiens,no,GEO,SRA603022,public,55B430EF91FD45723A75A0F83E487566,A1E7835851FA48F2F69C0DBF1E4E0640


(1650, 81)

In [3]:
# Extarct the Case Samples
Case=(MetaData["characteristics_ch1"]== "cell type: Tcells") & (MetaData["characteristics_ch1.3"]=="timepoint: 0") & (MetaData["characteristics_ch1.5"]=="group: case") & (MetaData["characteristics_ch1.1"]=="stimulation: unstim")
Case_samples = MetaData["Sample_ID"][Case]
case_samples = pd.DataFrame(Case_samples)
case_samples['Labels'] = 1
case_samples.head(2)
case_samples.shape

,Sample_ID,Labels
217,SRR5980958,1
220,SRR5980961,1


(40, 2)

In [4]:
# Extarct the Control Samples
Control=(MetaData["characteristics_ch1"]== "cell type: Tcells") & (MetaData["characteristics_ch1.3"]=="timepoint: 0") & (MetaData["characteristics_ch1.5"]=="group: control") & (MetaData["characteristics_ch1.1"]=="stimulation: unstim")
Control_samples = MetaData["Sample_ID"][Control]
control_samples = pd.DataFrame(Control_samples)
control_samples['Condition'] = 0
control_samples.head(2)
control_samples.shape

,Sample_ID,Condition
218,SRR5980959,0
219,SRR5980960,0


(73, 2)

In [5]:
# Extract Samples with labels
sample_id_map = pd.concat([control_samples, case_samples], axis = 0)
sample_id_map.head(2)
sample_id_map.shape

# Write the map
#sample_id_map.to_csv("MetaData_Map_case_control_samples.csv", index = False)

,Sample_ID,Condition,Labels
218,SRR5980959,0.0,NaN
219,SRR5980960,0.0,NaN


(113, 3)

In [6]:
# Import the Raw counts from HtSeq
raw_counts = pd.read_csv("Case_Control_Counts.csv")
raw_counts.shape
raw_counts = raw_counts.T.reset_index()
raw_counts.shape
raw_counts = raw_counts.rename(columns=raw_counts.iloc[0]).drop(raw_counts.index[0])
raw_counts.shape


(16507, 348)

(348, 16508)

(347, 16508)

In [7]:
raw_counts = raw_counts.rename(columns={'ensgene': 'Sample_ID'})
raw_counts.shape
raw_counts.head(2)
raw_counts.shape

(347, 16508)

,Sample_ID,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000286261,ENSG00000286264,ENSG00000286268,ENSG00000286522,ENSG00000286905,ENSG00000286920,ENSG00000287080,ENSG00000287542,ENSG00000287694,ENSG00000287725
1,SRR5980424,0,1292,987,257,105,66,1332,2011,102,...,1,0,0,0,0,0,0,0,0,4
2,SRR5980425,188,142,69,5,19,33,344,23,440,...,0,0,0,0,0,1,0,25,0,0


(347, 16508)

In [11]:
# Select the samples having labels
samples = pd.DataFrame(sample_id_map["Sample_ID"])
samples.shape
# Merge the frame
counts = pd.merge(samples, raw_counts, on = "Sample_ID")
counts.shape

(113, 1)

(111, 16508)

In [ ]:
# Transpose data for differential expression
counts = counts.T.reset_index()
counts.shape
counts = counts.rename(columns=counts.iloc[0]).drop(counts.index[0])
counts.shape
counts = counts.rename(columns={'Sample_ID': 'ensgene'})

In [15]:
counts = counts.T.reset_index()
counts.shape
counts

(16509, 113)

,index,0,1,2,3,4,5,6,7,8,...,102,103,104,105,106,107,108,109,110,111
0,index,index,0,1,2,3,4,5,6,7,...,101,102,103,104,105,106,107,108,109,110
1,0,Sample_ID,SRR5980959,SRR5980960,SRR5980962,SRR5980963,SRR5980965,SRR5980966,SRR5980968,SRR5980969,...,SRR5981118,SRR5981143,SRR5981166,SRR5981172,SRR5981211,SRR5981212,SRR5981214,SRR5981219,SRR5981240,SRR5981248
2,1,ENSG00000000005,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,ENSG00000000419,690,669,838,216,672,1021,1044,495,...,1147,1766,984,1353,1122,688,988,854,843,1457
4,3,ENSG00000000457,1140,847,1045,281,626,813,677,337,...,956,902,792,901,751,691,551,706,1082,1550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16504,16503,ENSG00000286920,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
16505,16504,ENSG00000287080,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
16506,16505,ENSG00000287542,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16507,16506,ENSG00000287694,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
counts = pd.read_csv("Counts.csv")

In [17]:
counts

,ensgene,SRR5980959,SRR5980960,SRR5980962,SRR5980963,SRR5980965,SRR5980966,SRR5980968,SRR5980969,SRR5980971,...,SRR5981118,SRR5981143,SRR5981166,SRR5981172,SRR5981211,SRR5981212,SRR5981214,SRR5981219,SRR5981240,SRR5981248
0,ENSG00000000005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ENSG00000000419,690,669,838,216,672,1021,1044,495,1027,...,1147,1766,984,1353,1122,688,988,854,843,1457
2,ENSG00000000457,1140,847,1045,281,626,813,677,337,675,...,956,902,792,901,751,691,551,706,1082,1550
3,ENSG00000000460,342,322,320,98,206,200,179,102,177,...,320,238,249,228,231,185,146,186,296,399
4,ENSG00000000938,503,202,142,10,64,107,63,45,372,...,181,77,48,73,212,195,89,23,1051,3147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16502,ENSG00000286920,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
16503,ENSG00000287080,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
16504,ENSG00000287542,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16505,ENSG00000287694,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
counts2 = counts.T.reset_index()

In [21]:
counts2 = counts2.rename(columns=counts2.iloc[0]).drop(counts2.index[0])


,ensgene,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000286261,ENSG00000286264,ENSG00000286268,ENSG00000286522,ENSG00000286905,ENSG00000286920,ENSG00000287080,ENSG00000287542,ENSG00000287694,ENSG00000287725
1,SRR5980959,0,690,1140,342,503,39,1164,1576,144,...,4,0,4,0,2,0,0,0,0,21
2,SRR5980960,0,669,847,322,202,78,842,1502,109,...,0,0,0,0,2,0,0,0,0,14
3,SRR5980962,0,838,1045,320,142,19,1036,1616,144,...,1,0,2,0,5,1,0,0,0,6
4,SRR5980963,0,216,281,98,10,65,389,385,25,...,1,0,0,0,6,0,0,0,0,5
5,SRR5980965,0,672,626,206,64,9,589,851,69,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,SRR5981212,0,688,691,185,195,9,630,991,111,...,1,0,0,0,2,0,0,0,0,7
108,SRR5981214,0,988,551,146,89,29,623,621,76,...,0,1,0,0,0,0,0,0,0,5
109,SRR5981219,0,854,706,186,23,41,582,984,101,...,1,0,0,0,0,0,0,0,0,0
110,SRR5981240,0,843,1082,296,1051,11,121,466,408,...,0,0,0,0,1,0,0,0,0,0


In [22]:
counts3 = pd.DataFrame(counts2["ensgene"])

In [26]:
counts2 = counts2.T

In [27]:
counts2

,1,2,3,4,5,6,7,8,9,10,...,102,103,104,105,106,107,108,109,110,111
ensgene,SRR5980959,SRR5980960,SRR5980962,SRR5980963,SRR5980965,SRR5980966,SRR5980968,SRR5980969,SRR5980971,SRR5980972,...,SRR5981118,SRR5981143,SRR5981166,SRR5981172,SRR5981211,SRR5981212,SRR5981214,SRR5981219,SRR5981240,SRR5981248
ENSG00000000005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000000419,690,669,838,216,672,1021,1044,495,1027,699,...,1147,1766,984,1353,1122,688,988,854,843,1457
ENSG00000000457,1140,847,1045,281,626,813,677,337,675,579,...,956,902,792,901,751,691,551,706,1082,1550
ENSG00000000460,342,322,320,98,206,200,179,102,177,165,...,320,238,249,228,231,185,146,186,296,399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000286920,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
ENSG00000287080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
ENSG00000287542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ENSG00000287694,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
counts2 = counts2.rename(columns=counts2.iloc[0]).drop(counts2.index[0])

In [30]:
counts2.to_csv("Counts.csv")

In [31]:
raw_counts = pd.read_csv("Case_Control_Counts.csv")

In [32]:
raw_counts.head(1)
raw_counts.shape

,ensgene,SRR5980424,SRR5980425,SRR5980430,SRR5980434,SRR5980437,SRR5980440,SRR5980443,SRR5980446,SRR5980449,...,SRR5981629,SRR5981630,SRR5981639,SRR5981645,SRR5981668,SRR5981670,SRR5981753,SRR5981754,SRR5981832,SRR5981970
0,ENSG00000000005,0,188,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(16507, 348)

In [33]:
raw_counts = raw_counts.rename(columns={'ensgene': 'Sample_ID'})

In [34]:
raw_counts.head(1)
raw_counts.shape

,Sample_ID,SRR5980424,SRR5980425,SRR5980430,SRR5980434,SRR5980437,SRR5980440,SRR5980443,SRR5980446,SRR5980449,...,SRR5981629,SRR5981630,SRR5981639,SRR5981645,SRR5981668,SRR5981670,SRR5981753,SRR5981754,SRR5981832,SRR5981970
0,ENSG00000000005,0,188,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(16507, 348)

In [35]:
raw_counts = raw_counts.T.reset_index()

In [36]:
raw_counts

,index,0,1,2,3,4,5,6,7,8,...,16497,16498,16499,16500,16501,16502,16503,16504,16505,16506
0,Sample_ID,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000286261,ENSG00000286264,ENSG00000286268,ENSG00000286522,ENSG00000286905,ENSG00000286920,ENSG00000287080,ENSG00000287542,ENSG00000287694,ENSG00000287725
1,SRR5980424,0,1292,987,257,105,66,1332,2011,102,...,1,0,0,0,0,0,0,0,0,4
2,SRR5980425,188,142,69,5,19,33,344,23,440,...,0,0,0,0,0,1,0,25,0,0
3,SRR5980430,0,745,846,315,232,3,1084,1567,106,...,3,2,3,0,4,0,0,0,0,14
4,SRR5980434,0,1059,847,289,20,42,1267,1610,137,...,5,0,0,0,1,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,SRR5981670,0,1468,975,306,442,74,236,550,424,...,2,0,0,0,0,0,0,0,0,0
344,SRR5981753,0,843,584,147,124,153,147,297,108,...,0,0,0,0,0,0,8,1,0,0
345,SRR5981754,0,1208,844,229,154,6,47,273,90,...,0,0,0,0,0,0,0,0,0,0
346,SRR5981832,0,2119,622,154,76,170,98,138,22,...,0,0,0,0,0,0,0,0,0,0


In [37]:
raw_counts = raw_counts.rename(columns=raw_counts.iloc[0]).drop(raw_counts.index[0])

In [38]:
raw_counts

,Sample_ID,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000286261,ENSG00000286264,ENSG00000286268,ENSG00000286522,ENSG00000286905,ENSG00000286920,ENSG00000287080,ENSG00000287542,ENSG00000287694,ENSG00000287725
1,SRR5980424,0,1292,987,257,105,66,1332,2011,102,...,1,0,0,0,0,0,0,0,0,4
2,SRR5980425,188,142,69,5,19,33,344,23,440,...,0,0,0,0,0,1,0,25,0,0
3,SRR5980430,0,745,846,315,232,3,1084,1567,106,...,3,2,3,0,4,0,0,0,0,14
4,SRR5980434,0,1059,847,289,20,42,1267,1610,137,...,5,0,0,0,1,0,0,0,0,4
5,SRR5980437,0,1394,953,330,103,16,1833,2111,144,...,1,1,0,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,SRR5981670,0,1468,975,306,442,74,236,550,424,...,2,0,0,0,0,0,0,0,0,0
344,SRR5981753,0,843,584,147,124,153,147,297,108,...,0,0,0,0,0,0,8,1,0,0
345,SRR5981754,0,1208,844,229,154,6,47,273,90,...,0,0,0,0,0,0,0,0,0,0
346,SRR5981832,0,2119,622,154,76,170,98,138,22,...,0,0,0,0,0,0,0,0,0,0


In [42]:
metadata = pd.read_csv("MetaData_Map.csv")
del metadata['Condition']

In [43]:
metadata.shape
metadata.head(1)

(111, 1)

,Sample_ID
0,SRR5980959


In [44]:
input_counts = pd.merge(metadata, raw_counts, on = "Sample_ID")

In [46]:
input_counts = input_counts.T.reset_index()

In [47]:
input_counts

,index,0,1,2,3,4,5,6,7,8,...,101,102,103,104,105,106,107,108,109,110
0,Sample_ID,SRR5980959,SRR5980960,SRR5980962,SRR5980963,SRR5980965,SRR5980966,SRR5980968,SRR5980969,SRR5980971,...,SRR5981118,SRR5981143,SRR5981166,SRR5981172,SRR5981211,SRR5981212,SRR5981214,SRR5981219,SRR5981240,SRR5981248
1,ENSG00000000005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ENSG00000000419,690,669,838,216,672,1021,1044,495,1027,...,1147,1766,984,1353,1122,688,988,854,843,1457
3,ENSG00000000457,1140,847,1045,281,626,813,677,337,675,...,956,902,792,901,751,691,551,706,1082,1550
4,ENSG00000000460,342,322,320,98,206,200,179,102,177,...,320,238,249,228,231,185,146,186,296,399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16503,ENSG00000286920,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
16504,ENSG00000287080,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
16505,ENSG00000287542,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
16506,ENSG00000287694,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
input_counts = input_counts.rename(columns=input_counts.iloc[0]).drop(input_counts.index[0])

In [50]:
input_counts.to_csv("Counts.csv", index = False)


In [53]:
input_counts = input_counts.dropna(axis = 1, how ='any')

In [61]:
input_counts = pd.read_csv("DEG_Case_Control_0_T_cell.csv")

In [57]:
input_counts = input_counts

In [62]:
input_counts.to_csv("input_counts_plot.csv")